In [108]:
from pandas.tseries.offsets import *
import pandas as pd
import numpy as np

In [11]:
path = {'order':'./training_data/order_data/order_data_{}', 
        'district':'./training_data/cluster_map/cluster_map'}

## Creating dictionary of District Info Table

In [31]:
district = pd.read_table(path['district'], header=None, index_col=0)
district = district[1].to_dict()

## Process order

In [156]:
day = '2016-01-01'
order = pd.read_table(path['order'].format(day), header=None, usecols=[1,3,6],
                      names=['driver', 'district_id', 'time'])

### Translating district hash to id

In [157]:
order['district_id'] = order['district_id'].apply(lambda x: district[x])

### Translating timestamp to slot

In [158]:
M = np.timedelta64(1, 'm')
day = pd.Timestamp(day)
order['time'] = pd.to_datetime(order['time'])

In [159]:
order['time_slot'] = (order['time'] - day) / M / 10 + 1
order['time_slot'] = order['time_slot'].astype(int)

In [162]:
order.describe()

,district_id,time_slot
count,501287.000000,501287.000000
mean,29.242442,73.997107
std,17.865127,40.038519
min,1.000000,1.000000
25%,12.000000,46.000000
50%,26.000000,79.000000
75%,48.000000,105.000000
max,66.000000,144.000000
